In [1]:
import gc
import os
import pandas as pd
from glob import glob
from itertools import combinations

In [13]:
sorted(glob(f"../data/interim/features/*/features*.hdf.compress"))

['../data/interim/features/app/features_2017-11-07_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app/features_2017-11-08_1700_1day_attributed.hdf.compress',
 '../data/interim/features/app/features_2017-11-08_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_channel/features_2017-11-07_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_channel/features_2017-11-08_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_device/features_2017-11-07_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_device/features_2017-11-08_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_ip/features_2017-11-07_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_ip/features_2017-11-08_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_os/features_2017-11-07_1700_1day_attributed2.hdf.compress',
 '../data/interim/features/app_os/features_2017-11-08_1700_1day_attributed2.hdf.co

In [23]:
sorted(glob(f"../data/interim/train_2017-11-0*00.csv"))

['../data/interim/train_2017-11-06_1400.csv',
 '../data/interim/train_2017-11-06_1500.csv',
 '../data/interim/train_2017-11-06_1600.csv',
 '../data/interim/train_2017-11-06_1700.csv',
 '../data/interim/train_2017-11-06_1800.csv',
 '../data/interim/train_2017-11-06_1900.csv',
 '../data/interim/train_2017-11-06_2000.csv',
 '../data/interim/train_2017-11-06_2100.csv',
 '../data/interim/train_2017-11-06_2200.csv',
 '../data/interim/train_2017-11-06_2300.csv',
 '../data/interim/train_2017-11-07_0000.csv',
 '../data/interim/train_2017-11-07_0100.csv',
 '../data/interim/train_2017-11-07_0200.csv',
 '../data/interim/train_2017-11-07_0300.csv',
 '../data/interim/train_2017-11-07_0400.csv',
 '../data/interim/train_2017-11-07_0500.csv',
 '../data/interim/train_2017-11-07_0600.csv',
 '../data/interim/train_2017-11-07_0700.csv',
 '../data/interim/train_2017-11-07_0800.csv',
 '../data/interim/train_2017-11-07_0900.csv',
 '../data/interim/train_2017-11-07_1000.csv',
 '../data/interim/train_2017-11-07

In [2]:
dtypes = {
    'ip': 'uint32',
    'app': 'uint16',
    'device': 'uint16',
    'os': 'uint16',
    'channel': 'uint16',
    'is_attributed': 'uint8'
}
to_read = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']

In [3]:
target_entities_init = ['app', 'device', 'os', 'channel', 'ip']
target_entities = []

for t in combinations(target_entities_init, 1):
    target_entities.append(t[0])
for t in combinations(target_entities_init, 2):
    target_entities.append(list(t))
# for t in combinations(target_entities_init, 3):
#     target_entities.append(list(t))
# for t in combinations(target_entities_init, 4):
#     target_entities.append(list(t))

target_entities.remove(['channel', 'ip'])

In [4]:
train_files = [
    '../data/interim/train_2017-11-07_1700.csv',
    '../data/interim/train_2017-11-07_1800.csv',
    '../data/interim/train_2017-11-07_1900.csv',
    '../data/interim/train_2017-11-07_2000.csv',
    '../data/interim/train_2017-11-07_2100.csv',
    '../data/interim/train_2017-11-07_2200.csv',
    '../data/interim/train_2017-11-07_2300.csv',
    '../data/interim/train_2017-11-08_0000.csv',
    '../data/interim/train_2017-11-08_0100.csv',
    '../data/interim/train_2017-11-08_0200.csv',
    '../data/interim/train_2017-11-08_0300.csv',
    '../data/interim/train_2017-11-08_0400.csv',
    '../data/interim/train_2017-11-08_0500.csv',
    '../data/interim/train_2017-11-08_0600.csv',
    '../data/interim/train_2017-11-08_0700.csv',
    '../data/interim/train_2017-11-08_0800.csv',
    '../data/interim/train_2017-11-08_0900.csv',
    '../data/interim/train_2017-11-08_1000.csv',
    '../data/interim/train_2017-11-08_1100.csv',
    '../data/interim/train_2017-11-08_1200.csv',
    '../data/interim/train_2017-11-08_1300.csv',
    '../data/interim/train_2017-11-08_1400.csv',
    '../data/interim/train_2017-11-08_1500.csv',
    '../data/interim/train_2017-11-08_1600.csv'
]

In [9]:
validation_files = [
    '../data/interim/train_2017-11-09_0400.csv',
    '../data/interim/train_2017-11-09_0500.csv',
    '../data/interim/train_2017-11-09_0600.csv',
]

In [5]:
def split(df, target_entity):
    if type(target_entity) == str:
        df[target_entity] = df.index
    else:
        df[target_entity[0]], df[target_entity[1]] = df.index.str.split('_', 1).str
        df[target_entity[0]] = df[target_entity[0]].astype(dtypes[target_entity[0]])
        df[target_entity[1]] = df[target_entity[1]].astype(dtypes[target_entity[1]])
    return df

In [6]:
def combine_features(train_files, features_prefix, feature_suffix):
    for train_file in train_files:
        df_train = pd.read_csv(train_file, usecols=to_read, dtype=dtypes)
        name, ext = os.path.splitext(os.path.basename(train_file))
        for target_entity in target_entities:
            target_entity_name = target_entity if type(target_entity) == str else "_".join(target_entity)
            feature_files = sorted(glob(f"../data/interim/features/{target_entity_name}/{features_prefix}*{feature_suffix}.hdf.compress"))
            assert len(feature_files) > 0
            for feature_file in feature_files:
                df_feature = pd.read_hdf(feature_file)
                df_feature = split(df_feature, target_entity)
                df_train = pd.merge(df_train, df_feature, how='left', left_on=target_entity, right_on=target_entity)
                del df_feature
                gc.collect()

        filename_output = f"../data/interim/combined/{name}_{feature_suffix}.hdf.compress"
        print(f"Writing {filename_output}")
        df_train.to_hdf(filename_output, 'train', mode='w', complib='blosc', fletcher32=True, complevel=9)
        del df_train
        gc.collect()

In [7]:
combine_features(train_files, 'features_2017-11-07_1700', 'attributed2')

Writing ../data/interim/combined/train_2017-11-07_1700_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_1800_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_1900_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_2000_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_2100_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_2200_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-07_2300_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-08_0000_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-08_0100_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-08_0200_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-08_0300_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-08_0400_attributed2.hdf.compress
Writing ../data/interim/combined/train_2

KeyboardInterrupt: 

In [10]:
combine_features(validation_files, 'features_2017-11-08_1700', 'attributed2')

Writing ../data/interim/combined/train_2017-11-09_0400_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-09_0500_attributed2.hdf.compress
Writing ../data/interim/combined/train_2017-11-09_0600_attributed2.hdf.compress


In [82]:
%%time
df = pd.read_hdf('../data/interim/combined/train_2017-11-07_1700_attributed2.hdf.compress')

CPU times: user 8.7 s, sys: 1.32 s, total: 10 s
Wall time: 10 s


In [32]:
sorted(glob(f"../data/interim/combined/train_2017-11-0*.hdf.compress"))

['../data/interim/combined/train_2017-11-07_1700_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_1800_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_1900_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2000_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2100_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2200_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2300_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0000_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0100_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0200_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0300_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0400_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0500_attributed.hdf.compress',
 '../data/interim/combine

In [2]:
train_feature_files = ['../data/interim/combined/train_2017-11-07_1700_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_1800_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_1900_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2000_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2100_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2200_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-07_2300_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0000_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0100_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0200_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0300_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0400_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0500_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0600_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0700_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0800_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_0900_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1000_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1100_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1200_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1300_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1400_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1500_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-08_1600_attributed.hdf.compress']

In [3]:
df_train = pd.concat([pd.read_hdf(f) for f in train_feature_files], ignore_index=True)

In [4]:
df_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed',
       'PERCENT_TRUE(clicks.is_attributed)_app_1day',
       'AVG_TIME_BETWEEN(clicks.click_time)_app_1day',
       'MODE(clicks.ip)_app_1day', 'MODE(clicks.device)_app_1day',
       'MODE(clicks.os)_app_1day', 'MODE(clicks.channel)_app_1day',
       'COUNT(clicks)_app_1day',
       'AVG_TIME_BETWEEN(clicks.click_time WHERE is_attributed = True)_app_1day',
       'MODE(clicks.ip WHERE is_attributed = True)_app_1day',
       'MODE(clicks.device WHERE is_attributed = True)_app_1day',
       'MODE(clicks.os WHERE is_attributed = True)_app_1day',
       'MODE(clicks.channel WHERE is_attributed = True)_app_1day',
       'MODE(clicks.HOUR(click_time))_app_1day',
       'MODE(clicks.HOUR(click_time) WHERE is_attributed = True)_app_1day',
       'COUNT(clicks WHERE is_attributed = True)_app_1day',
       'PERCENT_TRUE(clicks.is_attributed)_device_1day',
       'AVG_TIME_BETWEEN(clicks.click_time)_device_1day',
      

In [5]:
df_train.to_hdf('../data/processed/train_2017-11-07_1700_08_1600_attributed.hdf.compress', 'train', mode='w', complib='blosc', fletcher32=True)